In [1]:
import torch
from utils import quantise_tensor, dequantise_tensor, compute_quantisation_mse,get_tensor_memory_size

t = torch.rand((10, 100))
chunk_size = 256
t_q, scales, locations = quantise_tensor(t, chunk_size)
t_approx = dequantise_tensor(t_q, scales, locations, chunk_size)

mse = compute_quantisation_mse(t, t_approx)

print(f'Required memory: {get_tensor_memory_size(t)} vs {get_tensor_memory_size(t_q) + get_tensor_memory_size(scales) + get_tensor_memory_size(locations)}')

MSE: 0.0709843784570694
Norm: 18.278217315673828
RelativeMSE: 0.38836%
MSE: 0.07208407670259476
Norm: 18.6008243560791
RelativeMSE: 0.38753%
Required memory: 4000 vs 1032


In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM
from utils import quantise_model,get_model_memory_size, compute_quantisation_mse
import torch

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-uncased")

model = model.bert.encoder.layer[0]
input_tensor = torch.ones(1,10,768)

t = model.forward(input_tensor)[0]

get_model_memory_size(model)
model, parameter_mapping = quantise_model(model, chunk_size=128)
get_model_memory_size(model, parameter_mapping)

with torch.no_grad():
    tt = model.forward(input_tensor)[0]

compute_quantisation_mse(t, tt)

Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model size: 28351488 / bit | 28.35 / MB


Quantising model layers: 100%|██████████| 16/16 [00:06<00:00,  2.58it/s]


model size: 7530864 / bit | 7.53 / MB
MSE: 2.7140448093414307
Norm: 52.53758239746094
RelativeMSE: 5.16591%


tensor(2.7140, grad_fn=<SqrtBackward0>)